# OpenCV Eigenfaces for Face Recognition

This notebook is base on the course from Adrian Rosebrock from PyImageSearch. The original post can be found [here](https://pyimagesearch.com/2021/05/10/opencv-eigenfaces-for-face-recognition/).

In [6]:
# Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from skimage.exposure import rescale_intensity
from matplotlib import pyplot as plt
from imutils import build_montages, paths
import numpy as np
import argparse
import imutils
import time
import cv2
import os

# Helper functions

In [39]:
def load_face_data(path="./FacePhotosCleaned"):
    """Function to load the face dataset and return the cropped faces and labels"""
    faces = []
    labels = []
    for file in os.listdir(path):
        img = cv2.imread(path + "/" + file)
        img = cv2.resize(img, (200, 200))
        faces.append(img)
        labels.append(file.split(".")[0]) # get the name of the person from the file name
    return faces, labels

### Implementing Eigenfaces with OpenCV

In [46]:
# Load the data

(faces, labels) = load_face_data("/Users/diegosanmartin/Downloads/Computer Vision/FaceRecognition/FacePhotosCleaned")

# flatten all 2D faces into a 1D list of pixel intensities
pcaFaces = np.array([f.flatten() for f in faces])

# encode the string labels as integers
le = LabelEncoder()
labels = le.fit_transform(labels)

"""# construct our training and testing split
split = train_test_split(faces, pcaFaces, labels, test_size=0.25,
	stratify=labels, random_state=42)
(origTrain, origTest, trainX, testX, trainY, testY) = split"""

/var/folders/6s/16s1gbw10hv5zn6phb0s3b480000gn/T/ipykernel_55094/337928052.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pcaFaces = np.array([f.flatten() for f in faces])


'# construct our training and testing split\nsplit = train_test_split(faces, pcaFaces, labels, test_size=0.25,\n\tstratify=labels, random_state=42)\n(origTrain, origTest, trainX, testX, trainY, testY) = split'

In [53]:
for face in pcaFaces:
    print(len(face))

1442826
1140216
1963443
4082166
1096260
369603
2713203
594075
230187
453963
1474203
4969107
390963
892710
73947
175692
153228
578163
463347
7004352
2389668
4884528
157323
184512
297675
127308


In [51]:
# compute the PCA (eigenfaces) representation of the data, then
# project the training data onto the eigenfaces subspace
print("[INFO] creating eigenfaces...")
pca = PCA(
	svd_solver="randomized",
	n_components=150,
	whiten=True)
start = time.time()
trainX = pca.fit_transform(pcaFaces)
end = time.time()
print("[INFO] computing eigenfaces took {:.4f} seconds".format(
	end - start))

[INFO] creating eigenfaces...


ValueError: setting an array element with a sequence.

In [ ]:
# check to see if the PCA components should be visualized
if args["visualize"] > 0:
	# initialize the list of images in the montage
	images = []

	# loop over the first 16 individual components
	for (i, component) in enumerate(pca.components_[:16]):
		# reshape the component to a 2D matrix, then convert the data
		# type to an unsigned 8-bit integer so it can be displayed
		# with OpenCV
		component = component.reshape((62, 47))
		component = rescale_intensity(component, out_range=(0, 255))
		component = np.dstack([component.astype("uint8")] * 3)
		images.append(component)

	# construct the montage for the images
	montage = build_montages(images, (47, 62), (4, 4))[0]

	# show the mean and principal component visualizations
	# show the mean image
	mean = pca.mean_.reshape((62, 47))
	mean = rescale_intensity(mean, out_range=(0, 255)).astype("uint8")
	plt_imshow("Mean", mean)
	plt_imshow("Components", montage)

In [ ]:
# train a classifier on the eigenfaces representation
print("[INFO] training classifier...")
model = SVC(kernel="rbf", C=10.0, gamma=0.001, random_state=42)
model.fit(trainX, trainY)

# evaluate the model
print("[INFO] evaluating model...")
predictions = model.predict(pca.transform(testX))
print(classification_report(testY, predictions,
	target_names=le.classes_))

In [ ]:
# generate a sample of testing data
idxs = np.random.choice(range(0, len(testY)), size=10, replace=False)

# loop over a sample of the testing data
for i in idxs:
	# grab the predicted name and actual name
	predName = le.inverse_transform([predictions[i]])[0]
	actualName = le.classes_[testY[i]]

	# grab the face image and resize it such that we can easily see
	# it on our screen
	face = np.dstack([origTest[i]] * 3)
	face = imutils.resize(face, width=250)

	# draw the predicted name and actual name on the image
	cv2.putText(face, "pred: {}".format(predName), (5, 25),
		cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
	cv2.putText(face, "actual: {}".format(actualName), (5, 60),
		cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

	# display the predicted name  and actual name
	print("[INFO] prediction: {}, actual: {}".format(
		predName, actualName))

	# display the current face to our screen
	plt_imshow("Face", face)

For a detailed walkthrough of the concepts and code, be sure to refer to the full tutorial, [*OpenCV Eigenfaces for Face Recognition*](https://www.pyimagesearch.com/2021/05/10/opencv-eigenfaces-for-face-recognition/) published on 2021-05-10.

# Code License Agreement
```
Copyright (c) 2021 PyImageSearch.com

SIMPLE VERSION
Feel free to use this code for your own projects, whether they are
purely educational, for fun, or for profit. THE EXCEPTION BEING if
you are developing a course, book, or other educational product.
Under *NO CIRCUMSTANCE* may you use this code for your own paid
educational or self-promotional ventures without written consent
from Adrian Rosebrock and PyImageSearch.com.

LONGER, FORMAL VERSION
Permission is hereby granted, free of charge, to any person obtaining
a copy of this software and associated documentation files
(the "Software"), to deal in the Software without restriction,
including without limitation the rights to use, copy, modify, merge,
publish, distribute, sublicense, and/or sell copies of the Software,
and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be
included in all copies or substantial portions of the Software.
Notwithstanding the foregoing, you may not use, copy, modify, merge,
publish, distribute, sublicense, create a derivative work, and/or
sell copies of the Software in any work that is designed, intended,
or marketed for pedagogical or instructional purposes related to
programming, coding, application development, or information
technology. Permission for such use, copying, modification, and
merger, publication, distribution, sub-licensing, creation of
derivative works, or sale is expressly withheld.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS
BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN
ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```